In [22]:
import pandas as pd
import numpy as np
import pickle
import glob
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
path="/Users/CrystalZhang/Desktop/Global AI/SDG rating/master list/"

List A - companies

In [33]:
quandl=pd.read_csv(path+'sharadar.csv',index_col=0)
company_A= pd.DataFrame()
company_A["original_quandl"]=quandl["name"]
company_A["suffix"]=quandl["name"].apply(lambda x: [y.lower().capitalize() for y in x.split(' ')])
company_A["suffix"]=company_A["suffix"].apply(lambda x: list(filter(lambda y:y, map(lambda y:re.sub(r'\((.*)\)', '', y), x))))
company_A["suffix"]=company_A["suffix"].apply(lambda x: ' '.join(y for y in x))
company_A["suffix"]=company_A["suffix"].apply(str)
company_A=company_A.drop_duplicates()
company_A= company_A.dropna()
company_A= company_A.reset_index(drop=True)

In [34]:
len(company_A)

14869

List B - companies

In [35]:
IB=pd.read_csv(path+'IB_clean.csv')#if index_col=0 it will assign ticker to index col
company_B= pd.DataFrame()
company_B["original_IB"]=IB["LongName"]
company_B["suffix"]=IB['LongName'].apply(lambda x: x.split('-')[0])
company_B["suffix"]=company_B["suffix"].apply(lambda x: [y.lower().capitalize() for y in x.split(' ')])
company_B["suffix"]=company_B["suffix"].apply(lambda x: list(filter(lambda y:y, map(lambda y:re.sub(r'\((.*)\)', '', y), x))))
company_B["suffix"]=company_B["suffix"].apply(lambda x: ' '.join(y for y in x))
company_B["suffix"]=company_B["suffix"].apply(str)
company_B["suffix"]=company_B["suffix"].drop_duplicates()#cannot reset_index: nans are duplicated, are stacked at the bottom
#values are not nans are stacked on the top with wrong index
#company_B=company_B[company_B["suffix"].notnull()].reset_index(drop=True)#drop rows where suffix is null
company_B = company_B.dropna()
company_B = company_B.reset_index(drop=True)

In [36]:
len(company_B)

26011

In [7]:
#1.common words
list_words=company_A['suffix'].apply(str).values.tolist()+company_B['suffix'].apply(str).values.tolist()
string_words=' '.join(list_words)
list_word = string_words.split(' ')#from list_words to list_word: split words in each company name
df_word = pd.DataFrame({'list_word':list_word})
#df_word["list_word"].value_counts()
df_word['frequency'] = df_word.groupby('list_word')['list_word'].transform('count')
df_word=df_word.sort_values(by=['frequency'],ascending=False)
df_word=df_word.drop_duplicates(['list_word']).reset_index()
common_words=df_word['list_word'][:8].astype(str).values.tolist()#top 10 frequent words
# removelist=["Fund","Trust"]
# common_words = [word for word in common_words if word not in removelist]
common_words='\s'+"(?=\s)|\s".join(common_words)+"(?=\s)"#escape"." or it will match all character in regular expression
#common_words=common_words.replace(".","[.]")
common_words

'\\sInc(?=\\s)|\\sLtd(?=\\s)|\\sCorp(?=\\s)|\\sCo(?=\\s)|\\sGroup(?=\\s)|\\sHoldings(?=\\s)|\\sPlc(?=\\s)|\\sInternational(?=\\s)'

In [8]:
#2.Define N-grams
def ngrams(string, n=3):#n=5
    string = re.sub(r'[/]the|[,-./]',r' ', string)#\sOn(?=\s)|\sAviation(?=\s) #r" "
    string=re.sub(r'\s{1,}(?=[.,])|\s{1,}(?=\s)','',string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

#3.Tf_idf Matrix
company_names1=company_A['suffix']
company_names2=company_B['suffix']
company_names=pd.concat([company_names1,company_names2],ignore_index=True)
company_names='  '+company_names.str[:]+"  "
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(company_names)
tf_idf_matrix1=tf_idf_matrix[0:len(company_names1),:]
tf_idf_matrix2=tf_idf_matrix[len(company_names1):,:]

In [9]:
#4.Get Similarity and Matches
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [10]:
def get_matches_df(sparse_matrix,name_vector1,name_vector2,top=10000):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        n_matches = top
    else:
        n_matches = sparsecols.size
    
    left_side = np.empty([n_matches], dtype=object)
    right_side = np.empty([n_matches], dtype=object)
    similairity = np.zeros(n_matches)
    
    for index in range(0, n_matches):
        left_side[index] = name_vector1[sparserows[index]]
        right_side[index] = name_vector2[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
        
    
    return pd.DataFrame({'left_side': left_side,'right_side': right_side,'similairity': similairity})

In [11]:
matches = awesome_cossim_top(tf_idf_matrix1, tf_idf_matrix2.transpose(),100, 0.5)
matches_df = get_matches_df(matches,company_names1,company_names2,top=matches.count_nonzero())#top = 1000 if required 
#be careful about the index of the two company list

In [12]:
#First Half Result and Second Half Result
result=matches_df.sort_values(['similairity'],ascending=False)
result=result.drop_duplicates(['left_side'])
result1=result[result.similairity>0.75]#first half result
result2=result[result.similairity<=0.75]#second half result
#result2.reset_index(drop=True).head()
#result.to_csv('result.csv')

In [13]:
result1

,left_side,right_side,similairity
75516,Electronic Arts Inc,Electronic Arts Inc,1.000000
70811,Minerals Technologies Inc,Minerals Technologies Inc,1.000000
54474,Crossamerica Partners Lp,Crossamerica Partners Lp,1.000000
5503,William Hill Plc,William Hill Plc,1.000000
56507,Lilis Energy Inc,Lilis Energy Inc,1.000000
23926,Bp Midstream Partners Lp,Bp Midstream Partners Lp,1.000000
55433,Tesaro Inc,Tesaro Inc,1.000000
76424,Torotel Inc,Torotel Inc,1.000000
50025,Blue Hills Bancorp Inc,Blue Hills Bancorp Inc,1.000000
1611,Cyclacel Pharmaceuticals Inc,Cyclacel Pharmaceuticals Inc,1.000000


In [14]:
common_words2=df_word['list_word'][:150].astype(str).values.tolist()#top 120 frequent words 
#in case high similarity caused by common words
common_words2='\s'+"(?=\s)|\s".join(common_words2)+"(?=\s)"
common_words2=common_words2.replace(".","[.]")#escape"." or it will match all character in regular expression
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|'+common_words2,'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [15]:
result_names= pd.concat([result2.left_side, result2.right_side], ignore_index=True)
result_names1=result2['left_side'].reset_index(drop=True)#if don't reset_index, the get_matches_df can't work
result_names2=result2['right_side'].reset_index(drop=True)
result_names=' '+result_names.str[:]+" "#from two spaced to one
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)#analyzer: get_stop_words and build_tokenizer
tf_idf_matrix = vectorizer.fit_transform(result_names)

In [16]:
null_company=[]
for i in range(tf_idf_matrix.shape[0]):
    if len(tf_idf_matrix[i,:].toarray().nonzero()[1])<=12:
        # count_nonzero of four spaces and only five characters is 7 according to result check
        # too large will omit some different "XX technologies" to match
        null_company.append(result_names[i])
#null_company

In [17]:
result_clean= result_names.apply(lambda x: "".join(re.sub(r'[’]|'+common_words2,'',x))if x not in null_company else x)
#result_clean #save all null_company and clean other companies

In [18]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(result_clean)
tf_idf_matrix1=tf_idf_matrix[0:len(result_names1),:]
tf_idf_matrix2=tf_idf_matrix[len(result_names1):,:]

In [19]:
matches2=awesome_cossim_top(tf_idf_matrix1,tf_idf_matrix2.transpose(), 100, 0.75)#0.70
matches_df2 = get_matches_df(matches2,result_names1,result_names2,top=matches2.count_nonzero())

In [20]:
result2=matches_df2.sort_values(['similairity'],ascending=False).drop_duplicates()#[matches_df2['similairity']<0.999999]
#result2

In [21]:
result_all= pd.concat([result1, result2], ignore_index=True)
len(result_all)
# result_all['right_side'].unique().count()
# missing= list(set(tobacco_whole)-set(tobacco))
result_all

,left_side,right_side,similairity
0,Electronic Arts Inc,Electronic Arts Inc,1.000000
1,Minerals Technologies Inc,Minerals Technologies Inc,1.000000
2,Crossamerica Partners Lp,Crossamerica Partners Lp,1.000000
3,William Hill Plc,William Hill Plc,1.000000
4,Lilis Energy Inc,Lilis Energy Inc,1.000000
5,Bp Midstream Partners Lp,Bp Midstream Partners Lp,1.000000
6,Tesaro Inc,Tesaro Inc,1.000000
7,Torotel Inc,Torotel Inc,1.000000
8,Blue Hills Bancorp Inc,Blue Hills Bancorp Inc,1.000000
9,Cyclacel Pharmaceuticals Inc,Cyclacel Pharmaceuticals Inc,1.000000


In [46]:
result_all=result_all.rename(columns = {'left_side':'suffix'})
result_all=result_all.merge(company_A, on='suffix', how='left')
result_all=result_all.rename(columns = {'suffix':'suffix_quandl'})

result_all=result_all.rename(columns = {'right_side':'suffix'})
result_all=result_all.merge(company_B, on='suffix', how='left')
result_all=result_all.rename(columns = {'suffix':'suffix_IB'})

result_all.head()

,suffix_quandl,suffix_IB,similairity,original_quandl,original_IB
0,Electronic Arts Inc,Electronic Arts Inc,1.0,Electronic Arts Inc,ELECTRONIC ARTS INC
1,Minerals Technologies Inc,Minerals Technologies Inc,1.0,Minerals Technologies Inc,MINERALS TECHNOLOGIES INC
2,Crossamerica Partners Lp,Crossamerica Partners Lp,1.0,CrossAmerica Partners LP,CROSSAMERICA PARTNERS LP
3,William Hill Plc,William Hill Plc,1.0,William Hill plc,WILLIAM HILL PLC
4,Lilis Energy Inc,Lilis Energy Inc,1.0,Lilis Energy Inc,LILIS ENERGY INC


In [47]:
#result_all.to_csv('result_quandl_IB.csv')

In [52]:
#after manully check
result_all=pd.read_csv('result_quandl_IB_edit.csv',index_col=0)

In [53]:
result_all=result_all.drop_duplicates(subset=['suffix_IB']).reset_index(drop=True)

In [54]:
result_all

,suffix_quandl,suffix_IB,similairity,original_quandl,original_IB
0,Electronic Arts Inc,Electronic Arts Inc,1.000000,Electronic Arts Inc,ELECTRONIC ARTS INC
1,Minerals Technologies Inc,Minerals Technologies Inc,1.000000,Minerals Technologies Inc,MINERALS TECHNOLOGIES INC
2,Crossamerica Partners Lp,Crossamerica Partners Lp,1.000000,CrossAmerica Partners LP,CROSSAMERICA PARTNERS LP
3,William Hill Plc,William Hill Plc,1.000000,William Hill plc,WILLIAM HILL PLC
4,Lilis Energy Inc,Lilis Energy Inc,1.000000,Lilis Energy Inc,LILIS ENERGY INC
5,Bp Midstream Partners Lp,Bp Midstream Partners Lp,1.000000,BP Midstream Partners LP,BP MIDSTREAM PARTNERS LP
6,Tesaro Inc,Tesaro Inc,1.000000,TESARO Inc,TESARO INC
7,Torotel Inc,Torotel Inc,1.000000,Torotel Inc,TOROTEL INC
8,Blue Hills Bancorp Inc,Blue Hills Bancorp Inc,1.000000,Blue Hills Bancorp Inc,BLUE HILLS BANCORP INC
9,Cyclacel Pharmaceuticals Inc,Cyclacel Pharmaceuticals Inc,1.000000,Cyclacel Pharmaceuticals Inc,CYCLACEL PHARMACEUTICALS INC


In [55]:
quandl=quandl.rename(columns={"name":"original_quandl"})
IB=IB.rename(columns={"LongName":"original_IB"})

result_all=result_all.merge(quandl, on='original_quandl', how='outer')
result_all=result_all.merge(IB, on='original_IB', how='outer')

In [56]:
result_all.to_csv("IB_quandl_union.csv")

In [57]:
len(result_all)

40805